# Enantiomers
The data contains some enantiomeric monomers (n.b. if you consider the protecting group, you may be inclined to call them diastereomers instead. For reactivity, we would still expect them to behave nearly the same). We check how consistent these react.

In [ ]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import pandas as pd
from sklearn.metrics import accuracy_score

from src.util.db_utils import SynFermDatabaseConnection
from src.definitions import DATA_DIR

In [ ]:
con = SynFermDatabaseConnection()

In [ ]:
res = con.con.execute("SELECT initiator_long, monomer_long, terminator_long, long_name, l.binary_A, l.binary_B, l.binary_C, l.binary_D, l.binary_E, l.binary_F, l.binary_G, l.binary_H FROM experiments INNER JOIN main.labels l on experiments.id = l.experiment_id WHERE valid IS NULL OR valid NOT LIKE 'ERROR%';").fetchall()
df = pd.DataFrame(res, columns=["initiator", "monomer", "terminator", "long", "binary_A", "binary_B", "binary_C", "binary_D", "binary_E", "binary_F", "binary_G", "binary_H"])
df.head()

In [ ]:
enantiomer_pairs = [
["Mon001", "Mon087"],
["Mon003", "Mon078"],
["Mon011", "Mon088"],
["Mon013", "Mon074"],
["Mon014", "Mon090"],
["Mon015", "Mon076"],
["Mon016", "Mon096"],
["Mon017", "Mon075"],
["Mon019", "Mon091"],
["Mon020", "Mon077"],
]

In [ ]:
# check the accuracy for one of the enantiomers approximating the outcome for the other
for pair in enantiomer_pairs:
    exp1 = df.loc[df["monomer"] == pair[0]]
    exp2 = df.loc[df["monomer"] == pair[1]]
    both = pd.merge(exp1, exp2, on=["initiator", "terminator"], how="inner")
    print(pair)
    print(f"{len(both)} overlapping samples")
    for s in "ABCDEFGH":
        print(s, ":", accuracy_score(both[f"binary_{s}_x"], both[f"binary_{s}_y"]))

## Conclusion
For most products, formation is highly reproducible, especially for A, B, E, and F. H is the worst but still around 85-90% reproducible.
